1. Import Statements:

In [1]:
# Standard Python Modules:
import os
from datetime import datetime

# Progress Bar Generation:
from tqdm import tqdm

# Data Wrangling:
import numpy as np
import pandas as pd

# Statistical Tests & Modeling:
from scipy import stats
from statsmodels.tsa.api import ExponentialSmoothing
from pmdarima import auto_arima

# Machine Learning:

# Machine Learning Algorithm Implementations:
import xgboost as xgb

# Feature/Training Enhancements:
from sklearn.impute import KNNImputer

2. Changing Pandas Configuration:

In [2]:
pd.set_option('display.max_columns', None) # Disable of truncation of columns

3. User-defined function to compute SMAPE:

In [3]:
def compute_SMAPE(forecasted_values, actual_values):
    """
    function: compute_SMAPE
    description: Computes the SMAPE for the input arguments.

    args:
    1. forecasted_values
    2. actual_values
    """
    if forecasted_values.shape[0] == actual_values.shape[0]:
        n = forecasted_values.shape[0]
        smape = (
            np.sum(
                np.abs(forecasted_values - actual_values)
                / ((np.abs(forecasted_values) + np.abs(actual_values)) / 2)
            )
            / n
        )
        return smape

3. Reading the Training Dataset:

In [4]:
train = pd.read_csv("../data/train.csv")
train.columns = [feature_name.upper() for feature_name in train.columns]
train.FIRST_DAY_OF_MONTH = pd.to_datetime(train.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
train["YEAR"] = train.FIRST_DAY_OF_MONTH.dt.year
train["MONTH"] = train.FIRST_DAY_OF_MONTH.dt.month
train["STATE_FIPS_CODE"] = train.CFIPS.astype(str).apply(lambda x: x[:2])
train["COUNTY_FIPS_CODE"] = train.CFIPS.astype(str).apply(lambda x: x[1:])
del (
    train["COUNTY"],
    train["STATE"],
    train["ROW_ID"],
    train["ACTIVE"],
    train["FIRST_DAY_OF_MONTH"],
)
print(train.shape, "(ROWS, COLUMNS)")
train.head(3)

(122265, 6) (ROWS, COLUMNS)


,CFIPS,MICROBUSINESS_DENSITY,YEAR,MONTH,STATE_FIPS_CODE,COUNTY_FIPS_CODE
0,1001,3.007682,2019,8,10,001
1,1001,2.884870,2019,9,10,001
2,1001,3.055843,2019,10,10,001


Reading the Test Dataset:

In [5]:
test = pd.read_csv("../data/test.csv")
test.columns = [feature_name.upper() for feature_name in test.columns]
test.FIRST_DAY_OF_MONTH = pd.to_datetime(test.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
test["YEAR"] = test.FIRST_DAY_OF_MONTH.dt.year
test["MONTH"] = test.FIRST_DAY_OF_MONTH.dt.month
test["STATE_FIPS_CODE"] = test.CFIPS.astype(str).apply(lambda x: x[:2])
test["COUNTY_FIPS_CODE"] = test.CFIPS.astype(str).apply(lambda x: x[1:])
del test["ROW_ID"], test["FIRST_DAY_OF_MONTH"]
print(test.shape, "(ROWS, COLUMNS)")
test.head(3)

(25080, 5) (ROWS, COLUMNS)


,CFIPS,YEAR,MONTH,STATE_FIPS_CODE,COUNTY_FIPS_CODE
0,1001,2022,11,10,001
1,1003,2022,11,10,003
2,1005,2022,11,10,005


Reading the Census Data:

In [6]:
census_starter = pd.read_csv("../data/census_starter.csv")
census_starter.columns = [
    feature_name.upper() for feature_name in census_starter.columns
]
census_starter.head(3)

,PCT_BB_2017,PCT_BB_2018,PCT_BB_2019,PCT_BB_2020,PCT_BB_2021,CFIPS,PCT_COLLEGE_2017,PCT_COLLEGE_2018,PCT_COLLEGE_2019,PCT_COLLEGE_2020,PCT_COLLEGE_2021,PCT_FOREIGN_BORN_2017,PCT_FOREIGN_BORN_2018,PCT_FOREIGN_BORN_2019,PCT_FOREIGN_BORN_2020,PCT_FOREIGN_BORN_2021,PCT_IT_WORKERS_2017,PCT_IT_WORKERS_2018,PCT_IT_WORKERS_2019,PCT_IT_WORKERS_2020,PCT_IT_WORKERS_2021,MEDIAN_HH_INC_2017,MEDIAN_HH_INC_2018,MEDIAN_HH_INC_2019,MEDIAN_HH_INC_2020,MEDIAN_HH_INC_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,20.6,3.2,3.4,3.7,3.4,3.5,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,6.7,2.7,2.5,2.7,2.6,2.6,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0


Imputing the missing values in the Census Data:

In [7]:
imputed_census_starter = pd.DataFrame(
    KNNImputer(n_neighbors=5).fit_transform(census_starter),
    columns=list(census_starter.columns),
)
imputed_census_starter.head(3)

,PCT_BB_2017,PCT_BB_2018,PCT_BB_2019,PCT_BB_2020,PCT_BB_2021,CFIPS,PCT_COLLEGE_2017,PCT_COLLEGE_2018,PCT_COLLEGE_2019,PCT_COLLEGE_2020,PCT_COLLEGE_2021,PCT_FOREIGN_BORN_2017,PCT_FOREIGN_BORN_2018,PCT_FOREIGN_BORN_2019,PCT_FOREIGN_BORN_2020,PCT_FOREIGN_BORN_2021,PCT_IT_WORKERS_2017,PCT_IT_WORKERS_2018,PCT_IT_WORKERS_2019,PCT_IT_WORKERS_2020,PCT_IT_WORKERS_2021,MEDIAN_HH_INC_2017,MEDIAN_HH_INC_2018,MEDIAN_HH_INC_2019,MEDIAN_HH_INC_2020,MEDIAN_HH_INC_2021
0,76.6,78.9,80.6,82.7,85.5,1001.0,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317.0,58786.0,58731.0,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003.0,20.4,20.7,21.0,20.2,20.6,3.2,3.4,3.7,3.4,3.5,1.4,1.3,1.4,1.0,1.3,52562.0,55962.0,58320.0,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005.0,7.6,7.8,7.6,7.3,6.7,2.7,2.5,2.7,2.6,2.6,0.5,0.3,0.8,1.1,0.8,33368.0,34186.0,32525.0,34990.0,36422.0


Checking the dataset distribution before and after imputation:

In [8]:
pd.concat(
    [
        census_starter.mean().rename("ORIGINAL_MEAN"),
        imputed_census_starter.mean().rename("IMPUTED_MEAN"),
        census_starter.std().rename("ORIGINAL_SD"),
        imputed_census_starter.std().rename("IMPUTED_SD"),
    ],
    axis=1,
)

,ORIGINAL_MEAN,IMPUTED_MEAN,ORIGINAL_SD,IMPUTED_SD
PCT_BB_2017,69.920401,69.920401,9.702052,9.702052
PCT_BB_2018,72.690866,72.690866,9.255863,9.255863
PCT_BB_2019,75.398600,75.398600,8.846665,8.846665
PCT_BB_2020,78.543298,78.545672,8.250864,8.250623
PCT_BB_2021,80.539096,80.540535,7.889931,7.889087
CFIPS,30383.649268,30383.649268,15162.508374,15162.508374
PCT_COLLEGE_2017,13.813399,13.813399,5.586649,5.586649
PCT_COLLEGE_2018,14.005379,14.005379,5.630199,5.630199
PCT_COLLEGE_2019,14.240452,14.240452,5.689780,5.689780
PCT_COLLEGE_2020,14.631328,14.634157,5.776940,5.778196


In [9]:
del census_starter

In [10]:
print(train.shape)
train = pd.merge(left=train, right=imputed_census_starter, on=["CFIPS"], how="inner")
print(train.shape)
train.head(3)

(122265, 6)
(122265, 31)


,CFIPS,MICROBUSINESS_DENSITY,YEAR,MONTH,STATE_FIPS_CODE,COUNTY_FIPS_CODE,PCT_BB_2017,PCT_BB_2018,PCT_BB_2019,PCT_BB_2020,PCT_BB_2021,PCT_COLLEGE_2017,PCT_COLLEGE_2018,PCT_COLLEGE_2019,PCT_COLLEGE_2020,PCT_COLLEGE_2021,PCT_FOREIGN_BORN_2017,PCT_FOREIGN_BORN_2018,PCT_FOREIGN_BORN_2019,PCT_FOREIGN_BORN_2020,PCT_FOREIGN_BORN_2021,PCT_IT_WORKERS_2017,PCT_IT_WORKERS_2018,PCT_IT_WORKERS_2019,PCT_IT_WORKERS_2020,PCT_IT_WORKERS_2021,MEDIAN_HH_INC_2017,MEDIAN_HH_INC_2018,MEDIAN_HH_INC_2019,MEDIAN_HH_INC_2020,MEDIAN_HH_INC_2021
0,1001,3.007682,2019,8,10,001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317.0,58786.0,58731.0,57982.0,62660.0
1,1001,2.884870,2019,9,10,001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317.0,58786.0,58731.0,57982.0,62660.0
2,1001,3.055843,2019,10,10,001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317.0,58786.0,58731.0,57982.0,62660.0


In [11]:
print(test.shape)
test = pd.merge(left=test, right=imputed_census_starter, on=["CFIPS"], how="inner")
print(test.shape)
test.head(3)

(25080, 5)
(25080, 30)


,CFIPS,YEAR,MONTH,STATE_FIPS_CODE,COUNTY_FIPS_CODE,PCT_BB_2017,PCT_BB_2018,PCT_BB_2019,PCT_BB_2020,PCT_BB_2021,PCT_COLLEGE_2017,PCT_COLLEGE_2018,PCT_COLLEGE_2019,PCT_COLLEGE_2020,PCT_COLLEGE_2021,PCT_FOREIGN_BORN_2017,PCT_FOREIGN_BORN_2018,PCT_FOREIGN_BORN_2019,PCT_FOREIGN_BORN_2020,PCT_FOREIGN_BORN_2021,PCT_IT_WORKERS_2017,PCT_IT_WORKERS_2018,PCT_IT_WORKERS_2019,PCT_IT_WORKERS_2020,PCT_IT_WORKERS_2021,MEDIAN_HH_INC_2017,MEDIAN_HH_INC_2018,MEDIAN_HH_INC_2019,MEDIAN_HH_INC_2020,MEDIAN_HH_INC_2021
0,1001,2022,11,10,001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317.0,58786.0,58731.0,57982.0,62660.0
1,1001,2022,12,10,001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317.0,58786.0,58731.0,57982.0,62660.0
2,1001,2023,1,10,001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317.0,58786.0,58731.0,57982.0,62660.0


In [12]:
train.STATE_FIPS_CODE.nunique(), test.STATE_FIPS_CODE.nunique()

(49, 49)

In [13]:
print(train.shape)
train = pd.concat(
    [
        train,
        pd.get_dummies(
            train["STATE_FIPS_CODE"], prefix="STATE_FIPS_CODE_", drop_first=True
        ),
    ],
    axis=1,
)
del train["STATE_FIPS_CODE"]
print(train.shape)

(122265, 31)
(122265, 78)


In [14]:
print(test.shape)
test = pd.concat(
    [
        test,
        pd.get_dummies(
            test["STATE_FIPS_CODE"], prefix="STATE_FIPS_CODE_", drop_first=True
        ),
    ],
    axis=1,
)
del test["STATE_FIPS_CODE"]
print(test.shape)

(25080, 30)
(25080, 77)


In [15]:
total_data = pd.concat([
    train.assign(FILE_TYPE="TRAIN"), 
    test.assign(MICROBUSINESS_DENSITY=np.nan).assign(FILE_TYPE="TEST")
], axis=0)
total_data = total_data.sort_values(
    by=["CFIPS", "YEAR", "MONTH"], ascending=True
).reset_index(drop=True)

Implementation of Exponential Smoothing:

In [16]:
# Additive Triple Exponential Smoothing: (4 Seasonal Periods)
total_data["TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"] = np.nan
fips_codes = list(set(total_data["CFIPS"]))
for fips_code_index in tqdm(range(len(fips_codes))):
    cfip_ts = total_data.loc[
        total_data["CFIPS"]==fips_codes[fips_code_index], 
        "MICROBUSINESS_DENSITY"
    ].dropna().reset_index(drop=True)
    fit1 = ExponentialSmoothing(
    cfip_ts,
    seasonal_periods=4,
    trend="add",
    seasonal="add",
    use_boxcox=False,
    initialization_method="estimated",
    ).fit()
    forecasted_values = fit1.forecast(steps=8)
    total_data.loc[
        (total_data["CFIPS"]==fips_codes[fips_code_index])
        & (total_data["FILE_TYPE"]=="TEST"), 
        "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"
    ] = forecasted_values.to_list()

100%|██████████| 3135/3135 [01:43<00:00, 30.30it/s]


In [17]:
submission = pd.read_csv("../data/test.csv")
submission.columns = [feature_name.upper() for feature_name in submission.columns]
submission.FIRST_DAY_OF_MONTH = pd.to_datetime(submission.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
submission["YEAR"] = submission.FIRST_DAY_OF_MONTH.dt.year
submission["MONTH"] = submission.FIRST_DAY_OF_MONTH.dt.month
submission = submission.merge(
    right=total_data.loc[
        total_data["FILE_TYPE"]=="TEST", 
        ["CFIPS", "YEAR", "MONTH", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
    ],
    on=["CFIPS", "YEAR", "MONTH"],
    how="left"
)
submission = submission[
    ["ROW_ID", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
].rename(columns={
    "ROW_ID": "row_id",
    "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE": "microbusiness_density"
})

In [18]:
submission.to_csv(
    "../Submissions/" +
    "Holt-Winters Exponential Smoothing (Additive, 4 Seasonal Periods).csv",
     index=False
) # Submission Score: 1.8725 & Leaderboard Position: 1564

In [19]:
# Additive Triple Exponential Smoothing: (2 Seasonal Periods)
total_data["TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"] = np.nan
fips_codes = list(set(total_data["CFIPS"]))
for fips_code_index in tqdm(range(len(fips_codes))):
    cfip_ts = total_data.loc[
        total_data["CFIPS"]==fips_codes[fips_code_index], 
        "MICROBUSINESS_DENSITY"
    ].dropna().reset_index(drop=True)
    fit1 = ExponentialSmoothing(
    cfip_ts,
    seasonal_periods=2,
    trend="add",
    seasonal="add",
    use_boxcox=False,
    initialization_method="estimated",
    ).fit()
    forecasted_values = fit1.forecast(steps=8)
    total_data.loc[
        (total_data["CFIPS"]==fips_codes[fips_code_index])
        & (total_data["FILE_TYPE"]=="TEST"), 
        "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"
    ] = forecasted_values.to_list()

100%|██████████| 3135/3135 [01:42<00:00, 30.69it/s]


In [20]:
submission = pd.read_csv("../data/test.csv")
submission.columns = [feature_name.upper() for feature_name in submission.columns]
submission.FIRST_DAY_OF_MONTH = pd.to_datetime(submission.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
submission["YEAR"] = submission.FIRST_DAY_OF_MONTH.dt.year
submission["MONTH"] = submission.FIRST_DAY_OF_MONTH.dt.month
submission = submission.merge(
    right=total_data.loc[
        total_data["FILE_TYPE"]=="TEST", 
        ["CFIPS", "YEAR", "MONTH", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
    ],
    on=["CFIPS", "YEAR", "MONTH"],
    how="left"
)
submission = submission[
    ["ROW_ID", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
].rename(columns={
    "ROW_ID": "row_id",
    "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE": "microbusiness_density"
})

In [21]:
submission.to_csv(
    "../Submissions/" +
    "Holt-Winters Exponential Smoothing (Additive, 2 Seasonal Periods).csv",
     index=False
) # Submission Score: 1.6479 & Leaderboard Position: 1515

In [22]:
# Additive Triple Exponential Smoothing: (3 Seasonal Periods)
total_data["TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"] = np.nan
fips_codes = list(set(total_data["CFIPS"]))
for fips_code_index in tqdm(range(len(fips_codes))):
    cfip_ts = total_data.loc[
        total_data["CFIPS"]==fips_codes[fips_code_index], 
        "MICROBUSINESS_DENSITY"
    ].dropna().reset_index(drop=True)
    fit1 = ExponentialSmoothing(
    cfip_ts,
    seasonal_periods=3,
    trend="add",
    seasonal="add",
    use_boxcox=False,
    initialization_method="estimated",
    ).fit()
    forecasted_values = fit1.forecast(steps=8)
    total_data.loc[
        (total_data["CFIPS"]==fips_codes[fips_code_index])
        & (total_data["FILE_TYPE"]=="TEST"), 
        "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"
    ] = forecasted_values.to_list()

100%|██████████| 3135/3135 [01:43<00:00, 30.42it/s]


In [23]:
submission = pd.read_csv("../data/test.csv")
submission.columns = [feature_name.upper() for feature_name in submission.columns]
submission.FIRST_DAY_OF_MONTH = pd.to_datetime(submission.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
submission["YEAR"] = submission.FIRST_DAY_OF_MONTH.dt.year
submission["MONTH"] = submission.FIRST_DAY_OF_MONTH.dt.month
submission = submission.merge(
    right=total_data.loc[
        total_data["FILE_TYPE"]=="TEST", 
        ["CFIPS", "YEAR", "MONTH", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
    ],
    on=["CFIPS", "YEAR", "MONTH"],
    how="left"
)
submission = submission[
    ["ROW_ID", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
].rename(columns={
    "ROW_ID": "row_id",
    "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE": "microbusiness_density"
})

In [24]:
submission.to_csv(
    "../Submissions/" +
    "Holt-Winters Exponential Smoothing (Additive, 3 Seasonal Periods).csv",
     index=False
) # Submission Score: 1.8471  & Leaderboard Position: 1515

Auto-ARIMA:

In [37]:
# Auto-ARIMA: Univariate (No Constraints & No Seasonality)
total_data["TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"] = np.nan
fips_codes = list(set(total_data["CFIPS"]))
for fips_code_index in tqdm(range(len(fips_codes))):
    cfip_ts = total_data.loc[
        total_data["CFIPS"]==fips_codes[fips_code_index], 
        "MICROBUSINESS_DENSITY"
    ].dropna().reset_index(drop=True)
    arima_model = auto_arima(
        y=cfip_ts,
        test="adf",
        seasonal=False,
        stationary=False
    )
    forecasted_values = arima_model.predict(n_periods=8)
    total_data.loc[
        (total_data["CFIPS"]==fips_codes[fips_code_index])
        & (total_data["FILE_TYPE"]=="TEST"), 
        "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"
    ] = forecasted_values.to_list()

100%|██████████| 3135/3135 [32:49<00:00,  1.59it/s]


In [38]:
submission = pd.read_csv("../data/test.csv")
submission.columns = [feature_name.upper() for feature_name in submission.columns]
submission.FIRST_DAY_OF_MONTH = pd.to_datetime(submission.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
submission["YEAR"] = submission.FIRST_DAY_OF_MONTH.dt.year
submission["MONTH"] = submission.FIRST_DAY_OF_MONTH.dt.month
submission = submission.merge(
    right=total_data.loc[
        total_data["FILE_TYPE"]=="TEST", 
        ["CFIPS", "YEAR", "MONTH", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
    ],
    on=["CFIPS", "YEAR", "MONTH"],
    how="left"
)
submission = submission[
    ["ROW_ID", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
].rename(columns={
    "ROW_ID": "row_id",
    "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE": "microbusiness_density"
})

In [39]:
submission.to_csv(
    "../Submissions/" +
    "Auto ARIMA - Univariate (No Constraints & No Seasonality).csv",
     index=False
) # Submission Score:1.482   & Leaderboard Position: 1595

In [40]:
# Auto-ARIMA: Univariate (No Constraints & Seasonality)
total_data["TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"] = np.nan
fips_codes = list(set(total_data["CFIPS"]))
for fips_code_index in tqdm(range(len(fips_codes))):
    cfip_ts = total_data.loc[
        total_data["CFIPS"]==fips_codes[fips_code_index], 
        "MICROBUSINESS_DENSITY"
    ].dropna().reset_index(drop=True)
    arima_model = auto_arima(
        y=cfip_ts,
        test="adf",
        seasonal=True,
        stationary=False
    )
    forecasted_values = arima_model.predict(n_periods=8)
    total_data.loc[
        (total_data["CFIPS"]==fips_codes[fips_code_index])
        & (total_data["FILE_TYPE"]=="TEST"), 
        "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"
    ] = forecasted_values.to_list()

100%|██████████| 3135/3135 [21:05<00:00,  2.48it/s]


In [41]:
submission = pd.read_csv("../data/test.csv")
submission.columns = [feature_name.upper() for feature_name in submission.columns]
submission.FIRST_DAY_OF_MONTH = pd.to_datetime(submission.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
submission["YEAR"] = submission.FIRST_DAY_OF_MONTH.dt.year
submission["MONTH"] = submission.FIRST_DAY_OF_MONTH.dt.month
submission = submission.merge(
    right=total_data.loc[
        total_data["FILE_TYPE"]=="TEST", 
        ["CFIPS", "YEAR", "MONTH", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
    ],
    on=["CFIPS", "YEAR", "MONTH"],
    how="left"
)
submission = submission[
    ["ROW_ID", "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"]
].rename(columns={
    "ROW_ID": "row_id",
    "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE": "microbusiness_density"
})

In [42]:
submission.to_csv(
    "../Submissions/" +
    "Auto ARIMA - Univariate (No Constraints & with Seasonality).csv",
     index=False
) # Submission Score:1.4659   & Leaderboard Position: 1595

In [ ]:
# Auto ARIMA: Multivariate (No Constraints & No Seasonality)

Using Kruskal-Wallis test to determine the time-series that have the same distribution: Experimental

In [ ]:
# modeling_train = {}
# for cfips in set(train["CFIPS"]):
#     modeling_train[str(cfips)] = train.loc[train["CFIPS"]==cfips].copy()

# same_distributions = {}
# for cfips_1, df_1 in tqdm(modeling_train.items()):
#     if cfips_1 not in same_distributions.keys():
#         same_distributions[cfips_1] = []
#     for cfips_2, df_2 in modeling_train.items():
#         tstat, pval = stats.kruskal(
#             df_1['MICROBUSINESS_DENSITY'], 
#             df_2['MICROBUSINESS_DENSITY']
#         )
#         if pval<0.05:
#             same_distributions[cfips_1].append(cfips_2)

# final_combinations = {}
# covered_cfips = set()

# for cfip, combination in same_distributions.items():
#     if cfip not in covered_cfips:
#         covered_cfips.add(cfip)
#         covered_cfips = covered_cfips.union(combination)
#         final_combinations[cfip] = combination

In [ ]:
# total_data["TRIPLE_EXPONENTIAL_SMOOTHING_MULTIPLICATIVE"] = np.nan
# fips_codes = list(set(total_data["CFIPS"]))
# for fips_code_index in tqdm(range(len(fips_codes))):
#     cfip_ts = total_data.loc[
#         total_data["CFIPS"]==fips_codes[fips_code_index], 
#         "MICROBUSINESS_DENSITY"
#     ].dropna().reset_index(drop=True)
#     fit1 = ExponentialSmoothing(
#     cfip_ts,
#     seasonal_periods=4,
#     trend="add",
#     seasonal="add",
#     use_boxcox=False,
#     initialization_method="estimated",
#     ).fit()
#     forecasted_values = fit1.forecast(steps=8)
#     total_data.loc[
#         (total_data["CFIPS"]==fips_codes[fips_code_index])
#         & (total_data["FILE_TYPE"]=="TEST"), 
#         "TRIPLE_EXPONENTIAL_SMOOTHING_ADDITIVE"
#     ] = forecasted_values.to_list()

In [ ]:
# train = train.sort_values(by=["CFIPS", "YEAR", "MONTH"], ascending=True) # Sorting data in Chronological order for each CFIP.
# required_x_features = [colname for colname in train.columns if colname != "CFIPS"]
# window_size = 13
# X = []
# y = []
# for range_start_index in tqdm(range(train.shape[0]-window_size)):
#     range_end_index = range_start_index+window_size
#     range_data = train.iloc[range_start_index:range_end_index, :].copy()
#     y.append(range_data.loc[(range_end_index-1), "MICROBUSINESS_DENSITY"])
#     X.append(range_data.iloc[:-1,:].astype(float).values.tolist())
# X = np.array(X)
# y = np.array(y)
# X.shape, y.shape # Dimensions of the Training Data

# x_train, y_train = X[:73351], y[:73351]
# x_val, y_val = X[73351:83131], y[73351:83131]
# x_test, y_test = X[83131:], y[83131:]
# x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

# del (X, y)

# modeling_train = {}
# for cfips in set(train["CFIPS"]):
#     modeling_train[str(cfips)] = train.loc[train["CFIPS"]==cfips].copy()

# from scipy import stats
# from tqdm import tqdm

# same_distributions = {}
# for cfips_1, df_1 in tqdm(modeling_train.items()):
#     if cfips_1 not in same_distributions.keys():
#         same_distributions[cfips_1] = []
#     for cfips_2, df_2 in modeling_train.items():
#         tstat, pval = stats.kruskal(
#             df_1['MICROBUSINESS_DENSITY'], 
#             df_2['MICROBUSINESS_DENSITY']
#         )
#         if pval<0.05:
#             same_distributions[cfips_1].append(cfips_2)

# final_combinations = {}
# covered_cfips = set()

# for cfip, combination in same_distributions.items():
#     if cfip not in covered_cfips:
#         covered_cfips.add(cfip)
#         covered_cfips = covered_cfips.union(combination)
#         final_combinations[cfip] = combination